In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=db4c6ca9e63292af4891f2996f238b10784bdda2c97e94a85d4ba43d4d58abba
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 12991 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_2_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=917659, Tue Apr 14 16:36:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.601934558795190e+08 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.582619379002782e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.04758604e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.245667e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.379078545042247e+08 1.0e+00 2.33e+00  2e+00  2e+00 0:00.4
   27    513 2.900990621752784e+07 1.1e+00 2.05e+00  2e+00  2e+00 0:03.4
   59   1121 1.848968851201332e+07 1.1e+00 1.94e+00  2e+00  2e+00 0:07.5
  100   1900 -1.088861801612866e+07 1.2e+00 1.73e+00  2e+00  2e+00 0:12.7
  150   2850 -6.069298332603616e+07 1.3e+00 1.31e+00  1e+00  1e+00 0:18.7
  200   3800 -1.527406027466219e+08 1.3e+00 9.25e-01  9e-01  9e-01 0:24.7
  269   5111 -2.615255829504303e+08 1.3e+00 5.16e-01  5e-01  5e-01 0:32.7
  300   5700 -2.781700505818555e+08 1.3e+00 3.99e-01  4e-01  4e-01 0:36.3
  386   7334 -3.180660428547543e+08 1.3e+00 1.89e-01  2e-01  2e-01 0:46.3
  400   7600 -3.196338560511785e+08 1.3e+00 1.64e-01  2e-01  2e-01 0:48.0
  500   9500 -3.258885778960609e+08 1.3e+00 6.27e-02  6e-02  6e-02 0:59.7
  600  11400 -3.268653616535500e+08 1.4e+00 2.51e-02  2e-02  3e-02 1:11.4
  700  13300 -3.269826402212450e+08 1.4e+00 1.01e-02  9e-03  1e-02 1:23.3
  800  15200 -3.270091973872322e+08 1.4e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Tue Apr 14 16:58:25 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [ 1.33989184 -1.43436664 -0.32055517 -1.52712461  1.26831897 -3.80722518
 -3.02257653  1.32075761 ...]
std deviations: [2.43732433 2.43717402 2.43702689 2.4372035  2.43704729 2.43715739
 2.43729994 2.43707531 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=981194, Tue Apr 14 16:58:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.547132789416355e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.223499348015841e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

    3     57 1.144442840871506e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.7
   15    285 4.017678142848295e+07 1.0e+00 2.09e+00  2e+00  2e+00 0:03.8
   31    589 1.311674714580220e+07 1.1e+00 1.98e+00  2e+00  2e+00 0:07.9
   52    988 3.939899595046222e+06 1.1e+00 1.91e+00  2e+00  2e+00 0:13.0
   77   1463 -2.312869778468758e+06 1.2e+00 1.73e+00  2e+00  2e+00 0:19.1
  100   1900 -1.188041207153216e+07 1.2e+00 1.57e+00  2e+00  2e+00 0:25.2
  132   2508 -6.918880078569764e+07 1.2e+00 1.28e+00  1e+00  1e+00 0:33.4
  168   3192 -1.426056754072561e+08 1.2e+00 1.03e+00  1e+00  1e+00 0:42.5
  200   3800 -2.170251929030321e+08 1.2e+00 7.41e-01  7e-01  8e-01 0:50.2
  243   4617 -2.769977482700702e+08 1.3e+00 4.71e-01  5e-01  5e-01 1:01.2
  290   5510 -3.026421919428257e+08 1.3e+00 3.10e-01  3e-01  3e-01 1:13.3
  300   5700 -3.044832775939280e+08 1.3e+00 2.82e-01  3e-01  3e-01 1:15.9
  356   6764 -3.192901871123036e+08 1.3e+00 1.68e-01  2e-01  2e-01 1:29.9
  400   7600 -3.233305243944944e+08 1.3e+0

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Tue Apr 14 23:43:47 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [-0.3209569   2.79912203  4.46097043  1.7958246  -2.55691773  4.99414018
 -2.74756193 -3.04654608 ...]
std deviations: [2.43815937 2.43870262 2.43858287 2.43847051 2.43836892 2.43800384
 2.43888725 2.43822003 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1062472, Tue Apr 14 23:43:47 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.661549537370867e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 8.897030459408058e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.75427981e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.45167497e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.177790141148620e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   27    513 4.158942332469594e+07 1.1e+00 1.99e+00  2e+00  2e+00 0:03.5
   59   1121 4.082938340455435e+07 1.1e+00 1.77e+00  2e+00  2e+00 0:07.5
   99   1881 2.775244569628935e+07 1.2e+00 1.48e+00  1e+00  1e+00 0:12.6
  100   1900 2.325717741405769e+07 1.2e+00 1.47e+00  1e+00  1e+00 0:12.7
  155   2945 2.384284959510992e+07 1.2e+00 1.23e+00  1e+00  1e+00 0:19.8
  200   3800 2.205536886000334e+07 1.3e+00 1.03e+00  1e+00  1e+00 0:25.5
  272   5168 1.240170101510842e+07 1.3e+00 7.44e-01  7e-01  8e-01 0:34.5
  300   5700 1.313153436581148e+07 1.3e+00 7.00e-01  7e-01  7e-01 0:38.1
  385   7315 8.556432559448466e+06 1.4e+00 4.94e-01  5e-01  5e-01 0:49.2
  400   7600 9.046628894126281e+06 1.4e+00 4.62e-01  4e-01  5e-01 0:51.1
  500   9500 6.295534478448778e+06 1.4e+00 3.54e-01  3e-01  4e-01 1:03.5
  600  11400 4.322088519955963e+06 1.4e+00 2.70e-01  3e-01  3e-01 1:15.8
  700  13300 3.008982195352867e+06 1.5e+00 2.08e-01

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:200])
    SVM_Fun [i] = F2(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F2(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([50672204.6014525]), count=array([1])) 0.3195846940288272
SVM
ModeResult(mode=array([1.73187866e+08]), count=array([1])) 32604991.44516605
ELN
ModeResult(mode=array([3134512.0272758]), count=array([1])) 291022.73764659005
